In [173]:
%reset -f

# AIML CA1

## Import General Dependencies

In [ ]:
# Mathematical Dependencies
import numpy as np

# Data Manipulation Dependencies
import pandas as pd

# Graphing Dependencies
import matplotlib.pyplot as plt
import seaborn as sns

# Machine Learning Dependencies
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import scale, StandardScaler
from sklearn.pipeline import Pipeline

# Miscellaneous Dependencies
from typing import Callable, Dict # static typing

# Utility Functions
from utils.extraction import extract_attributes

In [ ]:
%matplotlib inline

In [ ]:
# Hide Warnings
from warnings import filterwarnings
filterwarnings(action='ignore')

## Utility Functions

## Part I

### Import Exclusive Dependencies

In [ ]:
# Machine Learning Models
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

# Classification Metrics
from sklearn.metrics import confusion_matrix

### Import Data

In [ ]:
def load_mushrooms() -> pd.DataFrame:
    # Extract raw content of ./data/agaricus-lepiota.names file
    metadata: str
    with open('./data/agaricus-lepiota.names') as f:
        metadata = f.read()

    # Extract attributes from metadata
    attrs = extract_attributes(metadata, r'7\. Attribute Information:.*\n((.|\n)*)8\. Missing')

    # Extract column names to be used for dataframe
    cols = attrs.keys()

    # Create the dataframe from ./data/agaricus-lepiota.data file,
    #   using column names derived from ./data/agaricus-lepiota.names file
    df = pd.read_csv(
        filepath_or_buffer='./data/agaricus-lepiota.data',
        sep=',',
        header=0,
        names=cols
    )

    # Expand attribute codes to their full definitions
    for col in cols:
        df[col].replace(to_replace=attrs[col] ,inplace=True)
    
    return df

In [ ]:
df = load_mushrooms()

#### Inspect Data

In [ ]:
# Inspect top 10 rows of the dataset
df.head(n=10)

#### Summarize Data

In [ ]:
# Inspect overview of the dataset
df.info()

In [ ]:
# Inspect statistics of the dataset
df.describe().transpose().sort_values(by='unique', ascending=False)

# Note that veil-type has only one value,
#   hence it is redundant
df.drop(labels='veil-type', axis=1, inplace=True)

### Pre-Processing

#### EDA

Target Distribution

In [ ]:
from utils.plotting import format_label
def plot_A(df: pd.DataFrame):
    ax = sns.countplot(data=df, x='class', palette='deep')
    ax.set_ylim(top=5000)
    ax.set_title(label='General Data Distribution')
    ax.set_ylabel(ylabel='Number of Records')
    ax.set_yticklabels(labels=format_label(
        ax.get_yticks() / 1000, lambda s: f'{round(s)}k'))
    ax.set_xlabel(xlabel='Type')
    total_count = df.shape[0]
    for p in ax.patches:
        x = p.get_x()
        y = p.get_height()
        ax.annotate(text=f'{y} ({y/total_count*100:.1f}%)',
                    xy=(x + 0.21, y + 70))
    return ax
ax_a = plot_A(df=df)
ax_a

Missing Values

In [ ]:
# Check for missing values
df.isna().sum(axis=0)

# Note that stalk-root has missing attributes (denoted as 'missing')
df['stalk-root'] = df['stalk-root'].str.replace(pat='missing', repl='unknown')

Correlation Between Attributes and Class

In [ ]:
def plot_B(df: pd.DataFrame):
    for i in df.drop(labels='class', axis=1).columns.values:
        fig, (corr_plot, freq_plot) = plt.subplots(ncols=2, figsize=(14, 6))
        ct = pd.crosstab(index=df['class'], columns=df[i])
        distr = df.groupby(i).count().iloc[:,0]
        proportion = (ct.iloc[1] - ct.iloc[0]) / distr
        corr = pd.DataFrame(proportion.reset_index())
        sns.barplot(data=corr, x=i, y=0, ax=corr_plot, color='grey')
        sns.countplot(data=df.sort_values(by=i), x=i, hue='class', ax=freq_plot, palette='turbo')
        fig.suptitle(t=f'{i.upper()}')
        corr_plot.set_title(label='Correlation (chi2-based)')
        corr_plot.set_ylim((-1.1, 1.1))
        corr_plot.set_ylabel(ylabel='Correlation')
        corr_plot.set_xticklabels(labels=corr_plot.get_xticklabels(), rotation=30)
        freq_plot.set_title(label=f'Frequency Distribution')
        freq_plot.set_xticklabels(labels=freq_plot.get_xticklabels(), rotation=30)
plot_B(df)

#### Feature Selection

In [ ]:
# (One-Hot) Encode the dataset (categorical -> binary)
df_ohe = pd.get_dummies(data=df, drop_first=True)

# Get correlation between top 10 factors and Class
df_ohe.corr()['class_poisonous'].drop(labels='class_poisonous').sort_values(key=lambda x: np.abs(x), ascending=False).head(n=10)

In [ ]:
# Chi2-based feature selection
from sklearn.feature_selection import chi2, SelectKBest

# Get top 10 factors that are correlated with Class
mask = SelectKBest(score_func=chi2, k=10).fit(X=df_ohe.drop(labels='class_poisonous', axis=1), y=df_ohe[['class_poisonous']])
good_preds = df_ohe.drop(labels='class_poisonous', axis=1).columns.values[mask.get_support()]
good_preds

### Data Partitioning

In [ ]:
# Split the dataset into training and test sets
X = df_ohe.drop(labels='class_poisonous', axis=1)
y = df_ohe['class_poisonous']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

### Algorithm Selection & Hyper-Parameter Tuning

In [ ]:
# Import candidate classification algorithms
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.base import BaseEstimator

class DummyEstimator(BaseEstimator):
    def fit(self): pass
    def score(self): pass

# pipeline = Pipeline(steps=[
#     ('scaler', StandardScaler()),
#     ('clf', DummyEstimator())
# ])

# params = [
#     {
#         'clf': [KNeighborsClassifier()],
#         'clf__n_neighbors': np.arange(start=4, stop=10)
#     },
#     {
#         'clf': [LogisticRegression(solver='newton-cg')],
#         'clf__C': np.logspace(-1, 2, 3)
#     },
#     {
#         'clf': [MultinomialNB()]
#     },
#     {
#         'clf': [SVC()],
#         'clf__C': np.logspace(-1, 2, 3)
#     },
#     {
#         'clf': [DecisionTreeClassifier()],
#         'clf__max_depth': [10, 20, 30]
#     }
# ]

# cv = GridSearchCV(estimator=pipeline, param_grid=params, cv=5)
# cv.fit(X=X, y=y)
# import pickle
# pickle.dump(obj=cv, file=open("./models/grid_search_clf.p", "wb"))
# print(cv.best_params_)
# print(cv.best_score_)
# print(cv.best_estimator_)
# pd.DataFrame(data=cv.cv_results_)

In [ ]:
import pickle
mod = pickle.load(file=open('./models/grid_search_clf.p', 'rb'))
df_results = pd.DataFrame(mod.cv_results_)
# df_results.to_csv('./tmp/a.csv')
df_results

In [ ]:
t_d = df_results.copy()
t_d['param_clf'].astype('str')
# ddd = t_d.groupby('param_clf').mean()
# try_out = ddd.reset_index().melt(id_vars='param_clf', var_name='test', value_name='s')
# try_out['test'] = try_out['test'].str.slice(5, 6).astype(int)
# try_out['test'] = try_out['test'].str.extract(pat=r'*([\d])*', expand=False)
# sns.lineplot(data=try_out, x='test', y='s', hue='param_clf')
# try_out

fig, ax = plt.subplots(ncols=5, sharey=True, figsize=(12, 8))
hyp = ['param_clf__n_neighbors', 'param_clf__C', None, 'param_clf__C', 'param_clf__max_depth']
for i, est in enumerate(pd.unique(t_d['param_clf'])):
    stuff = t_d[t_d['param_clf'] == est].melt(id_vars=['param_clf', 'param_clf__n_neighbors', 'param_clf__C', 'param_clf__max_depth'], value_vars=['split0_test_score', 'split1_test_score', 'split2_test_score', 'split3_test_score', 'split4_test_score'], var_name='test', value_name='score')
    stuff['test'] = stuff['test'].str.extract(pat='([\\d])', expand=False)
    stuff['test'] = stuff['test'].astype(int)
    stuff['test'] += 1
    stuff.dropna(axis=1, inplace=True)
    ax[i].set_ylim((0.5, 1.2))
    ax[i].set_title(est)
    ax[i].set_xticks(ticks=range(1, 6))
    sns.lineplot(data=stuff, x='test', y='score', hue=hyp[i], ax=ax[i], palette='muted')

### Model Training

Fit the data to the most consistent algorithm, using the best performing hyperparameters

In [ ]:
clf = LogisticRegression(C=3.16)
clf.fit(X=X_train, y=y_train)

### Model Scoring

Use the model to generate predictions

In [ ]:
y_pred = clf.predict(X=X_test)
y_pred

### Model Evaluation

Evaluate the performance of the model based on common metrics

In [ ]:
# Import model evaluation dependencies
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
# Classification summary
print(classification_report(y_true=y_test, y_pred=y_pred, target_names=['edible', 'poisonous']))

# Confusion matrix
print('\n', pd.DataFrame(data=confusion_matrix(y_true=y_test, y_pred=y_pred, labels=[0, 1]), index=['Actual Edible', 'Actual Poisonous'], columns=['Predicted Edible', 'Predicted Poisonous']), '\n\n', sep='')

# Print target distribution in y_test
print(y_test.groupby(y_test).count())

## Part II

### Import Exclusive Dependencies

In [ ]:
# Machine Learning Models (Regression)
from sklearn.linear_model import LinearRegression, BayesianRidge, Lasso, Ridge, ElasticNet

### Import Data

In [ ]:
# 
df2 = pd.read_csv('./data/kc_house_data.csv')

#### Inspect Data

In [ ]:
df2.head()

In [ ]:
df2.corr()

#### Summarize Data

In [ ]:
df2.info()

In [ ]:
df2.describe().transpose().round(2)

### Pre-Processing

#### EDA

In [ ]:
# Check for missing values
df2.isna().sum(axis=0)

In [ ]:
def plot_C():
    fig, ax = plt.subplots(figsize=(10, 8))
    sns.heatmap(data=df2.corr(), cmap='RdBu', vmin=-1, vmax=1, ax=ax)
plot_C()

Distribution of Individual Variables

In [ ]:
def plot_D():
    for i in df2.columns.values:
        if df2[i].dtype.kind in 'biufc':
            fig, (hst, bxp) = plt.subplots(ncols=2)
            sns.histplot(data=df2, x=i, ax=hst)
            sns.boxplot(data=df2, y=i, ax=bxp)
plot_D()

In [ ]:
positively_skewed = ['price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'sqft_above', 'sqft_basement', 'sqft_living15', 'sqft_lot15']

def plot_E(df: pd.DataFrame):
    for i in positively_skewed:
        perc = df[df[i] == 0].shape[0] / df.shape[0] * 100
        print(f'Percentage ({i}): {perc}%')
        df3 = df[df[i] > 0] if perc < 0.1 else df[[i]].replace(0, 1)
        fig, (bef, aft) = plt.subplots(ncols=2)
        sns.histplot(data=df, x=i, ax=bef)
        sns.histplot(data=np.log2(df3[i]), ax=aft)
plot_E(df2)

In [ ]:
df2.corr()['price'].sort_values(key=lambda x: np.abs(x), ascending=False).drop(['lat', 'long', 'price'])

In [ ]:
print(pd.unique(df2['id']).size, df2.count()['id'])
df2.drop(labels='id', axis=1, inplace=True)

In [ ]:
print(pd.unique(df2['zipcode']).size, df2.count()['zipcode'])
sns.relplot(data=df2, x='zipcode', y='price')
df2.drop(labels='zipcode', axis=1, inplace=True)

In [ ]:
fig, ax = plt.subplots(nrows=2, ncols=4)
d = np.array(ax).reshape((-1))
for i, t in enumerate(['waterfront', 'floors', 'yr_renovated', 'sqft_lot', 'sqft_lot15', 'yr_built', 'condition']):
    sns.scatterplot(data=df2, x=t, y='price', ax=d[i])

In [ ]:
# df3 = df2.drop(labels=['waterfront', 'floors', 'yr_renovated', 'yr_built', 'condition'])

In [ ]:
def plot_B(df: pd.DataFrame):
    top_features = df.corr()['price'].sort_values(key=lambda x: np.abs(x), ascending=False).drop(['lat', 'long', 'price'])[:9].index.values
    fig, ax = plt.subplots(nrows=3, ncols=3, figsize=(8, 6))
    axs = np.array(ax).reshape((-1))
    for i, x in enumerate(top_features):
        sns.scatterplot(data=df, x=x, y='price', ax=axs[i])
    return fig
f = plot_B(df2)
f

In [ ]:
df2['date'] = pd.to_datetime(arg=df2['date'], yearfirst=True)

In [ ]:
sns.scatterplot(data=df2, x='sqft_living', y='price')

In [ ]:
# sns.lineplot(data=df2, x='grade', y='price')
# sns.scatterplot(data=df2, x='grade', y='price')


In [ ]:
sns.boxplot(data=df2, x='bedrooms', y='price')

In [ ]:
sns.boxplot(data=df_tmp, x='bedrooms', y='price')

In [ ]:
# Check for outliers
outliers2 = df2[df2['']]

#### Feature Engineering

#### Feature Selection

In [ ]:
df2['date']

In [ ]:
df2_corr = df2.corr()
df2_corr.drop(labels=df2_corr.columns[df2_corr.columns != 'price'].values, axis=1).drop(labels='price', axis=0)['price'].sort_values(ascending=False)

### Data Partitioning

In [ ]:
X2 = df2[['sqft_living', 'grade', 'sqft_above', 'sqft_living15', 'bathrooms', 'view', 'sqft_basement', 'bedrooms', 'waterfront']]
y2 = df2['price']
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.2)
from sklearn.preprocessing import RobustScaler, normalize
#! delete !#
pip = Pipeline(steps=[
    ('scaler', normalize),
    ('linreg', LinearRegression())
])
model2 = LinearRegression()
pip.fit(X=X2_train, y=y2_train)
print(pip.score(X2_train, y2_train))
print(pip.score(X2_test, y2_test))

### Model Training

In [ ]:
model2 = LinearRegression()
model2.fit(X=X2_train, y=y2_train)
print(model2.score(X2_train, y2_train))
print(model2.score(X2_test, y2_test))

In [ ]:
model2 = BayesianRidge()
model2.fit(X=X2_train, y=y2_train)
print(model2.score(X2_train, y2_train))
print(model2.score(X2_test, y2_test))

In [ ]:
df_another = df2.copy()
positively_skewed = ['price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'sqft_above', 'sqft_living15', 'sqft_lot15']
for o in positively_skewed:
    df_another = df_another[df_another[o] > 0]

dfp = df_another['price']
df_another = np.log2(df_another[['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'sqft_above', 'sqft_living15', 'sqft_lot15']])

In [ ]:
print(df_another.shape)
print(df2.shape)

In [ ]:
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.preprocessing import QuantileTransformer



# expt - use df_another
X3_train, X3_test, y3_train, y3_test = train_test_split(df_another, dfp, test_size=0.2)

u = Pipeline(steps=[
    ('reg', Lasso())
])

params_ = [
    {
        'reg': [Lasso(), Ridge(), ElasticNet()],
        'reg__alpha': np.linspace(0, 2, 5)
    }
]

pd.DataFrame(GridSearchCV(u, params_).fit(X3_train, y3_train).cv_results_)

### Model Scoring

### Model Evaluation

## Conclusions